# Building a data analysis company with Group Chat

In [ ]:
! pip install "pyautogen[teachable]"

In [ ]:
## import required libraries
import os
from autogen import UserProxyAgent
from autogen import ConversableAgent  # As an example

load_dotenv() # load environment variables from .env file

In [ ]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## The task!

In [ ]:
task = "Write a detailed weather analysis report for New York City "\
"over the past week. Today's date is 2024-04-23."

## Create a group chat

This group chat will include these agents:

- User_proxy or Admin: to allow the user to comment on the report and ask the writer to refine it.
- Planner: to determine relevant information needed to complete the task.
- Engineer: to write code using the defined plan by the planner.
- Executor: to execute the code written by the engineer.
- Writer: to write the report.

In [ ]:
## user or admin agent
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the weather analysis report.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [ ]:
## planner agent
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what weather information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "weather information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

In [ ]:
## engineer agent
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code to fetch and analyze weather data "
    "based on the plan provided by the planner.",
)

In [ ]:
## executor agent
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the weather data results.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [ ]:
## writer agent
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Weather Report Writer. "
    "Please write weather analysis reports in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your report.",
    description="Writer. "
    "Write weather analysis reports based on the code execution results and take "
    "feedback from the admin to refine the report."
)